In [4]:
import numpy as np
from collections import defaultdict

from cortex import db

pycortex_subject = "sub-01"
rois_npz_fn = '/Users/uriel/Downloads/{}_mmp_atlas_rois.npz'.format(pycortex_subject)
data = np.load(rois_npz_fn)

# ---- Load flat surface (merge=True to match ROI vertices) ----
fl_pts, fl_polys = db.get_surf(pycortex_subject, "flat", merge=True, nudge=True)
lh_pts, _ = db.get_surf(pycortex_subject, "flat")[0]
n_lh = len(lh_pts)
n_rh = len(fl_pts) - n_lh


def build_adjacency(polys, n_vertices, offset=0):
    adj = [set() for _ in range(n_vertices)]

    for a, b, c in polys:
        if a < offset or b < offset or c < offset:
            continue
        if a >= offset + n_vertices or b >= offset + n_vertices or c >= offset + n_vertices:
            continue

        a -= offset
        b -= offset
        c -= offset

        adj[a].update([b, c])
        adj[b].update([a, c])
        adj[c].update([a, b])

    return adj


# adjacency per hemisphere
adj_lh = build_adjacency(fl_polys, n_lh, offset=0)
adj_rh = build_adjacency(fl_polys, n_rh, offset=n_lh)


def connected_components(mask, adj):
    verts = np.where(mask)[0]
    visited = set()
    comps = []

    for v in verts:
        if v in visited:
            continue
        stack = [v]
        visited.add(v)
        size = 0
        while stack:
            cur = stack.pop()
            size += 1
            for nb in adj[cur]:
                if nb not in visited and mask[nb]:
                    visited.add(nb)
                    stack.append(nb)
        comps.append(size)

    comps.sort(reverse=True)
    return comps


results = []

for roi_name in data.files:
    mask_full = data[roi_name]

    mask_lh = mask_full[:n_lh].astype(bool)
    mask_rh = mask_full[n_lh:].astype(bool)

    comps_lh = connected_components(mask_lh, adj_lh) if mask_lh.sum() else []
    comps_rh = connected_components(mask_rh, adj_rh) if mask_rh.sum() else []

    def stats(comps):
        if len(comps) == 0:
            return 0, 0, 0.0
        total = sum(comps)
        largest = comps[0]
        ratio = largest / total
        return len(comps), total, ratio

    ncomp_lh, total_lh, ratio_lh = stats(comps_lh)
    ncomp_rh, total_rh, ratio_rh = stats(comps_rh)

    results.append({
        "roi": roi_name,
        "ncomp_lh": ncomp_lh,
        "total_lh": total_lh,
        "ratio_lh": ratio_lh,
        "ncomp_rh": ncomp_rh,
        "total_rh": total_rh,
        "ratio_rh": ratio_rh,
        "comps_lh": comps_lh[:10],
        "comps_rh": comps_rh[:10],
    })


results_sorted = sorted(results, key=lambda x: min(
    x["ratio_lh"] if x["ratio_lh"] else 1,
    x["ratio_rh"] if x["ratio_rh"] else 1
))

for r in results_sorted[:50]:
    print(
        f"{r['roi']:30s} | "
        f"LH comps={r['ncomp_lh']:3d} ratio={r['ratio_lh']:.3f} | "
        f"RH comps={r['ncomp_rh']:3d} ratio={r['ratio_rh']:.3f} | "
        f"topLH={r['comps_lh']} topRH={r['comps_rh']}"
    )

H                              | LH comps=1385 ratio=0.001 | RH comps=1107 ratio=0.001 | topLH=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] topRH=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
EC                             | LH comps=739 ratio=0.001 | RH comps=569 ratio=0.002 | topLH=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] topRH=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
PreS                           | LH comps=654 ratio=0.050 | RH comps=446 ratio=0.036 | topLH=[35, 19, 1, 1, 1, 1, 1, 1, 1, 1] topRH=[17, 7, 1, 1, 1, 1, 1, 1, 1, 1]
PeEc                           | LH comps=639 ratio=0.525 | RH comps=846 ratio=0.212 | topLH=[706, 1, 1, 1, 1, 1, 1, 1, 1, 1] topRH=[247, 48, 26, 1, 1, 1, 1, 1, 1, 1]
RSC                            | LH comps=475 ratio=0.310 | RH comps=452 ratio=0.307 | topLH=[299, 147, 47, 1, 1, 1, 1, 1, 1, 1] topRH=[292, 208, 1, 1, 1, 1, 1, 1, 1, 1]
PHA1                           | LH comps=415 ratio=0.413 | RH comps=310 ratio=0.464 | topLH=[291, 1, 1, 1, 1, 1, 1, 1, 1, 1] topRH=[267, 1, 1, 1, 1, 1, 1, 1, 1, 1]
33pr      

In [3]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
import cortex

def flatten_surf(surf_data):
    pts, polys = surf_data

    # 1) pts
    if isinstance(pts, tuple):
        pts_lh, pts_rh = pts
        pts = np.vstack([pts_lh, pts_rh])
    else:
        pts = np.asarray(pts)

    # 2) polys
    if isinstance(polys, tuple):
        polys_lh, polys_rh = polys
        polys = np.vstack([polys_lh, polys_rh + len(pts_lh)])
    else:
        polys = np.asarray(polys)

    return pts, polys

def infer_nlh(polys):
    """
    Estimation robuste de n_lh même si les indices ont des trous.
    """
    idx = np.unique(polys.flatten())
    idx.sort()

    gaps = np.diff(idx)
    # gaps positions
    best_score = -1
    best_split = None

    for i, g in enumerate(gaps):
        if g < 2:  # ignore tiny gaps
            continue
        left = i + 1
        right = len(idx) - left
        score = left * right * g  # maximize balance + gap size
        if score > best_score:
            best_score = score
            best_split = idx[i] + 1

    if best_split is None:
        raise RuntimeError("Impossible de trouver le split LH/RH")

    return int(best_split)

def mesh_adjacency(polys, n_vertices):
    row, col = [], []
    for a, b, c in polys:
        row += [a, b, b, c, c, a]
        col += [b, a, c, b, a, c]
    data = np.ones(len(row), dtype=np.int8)
    return csr_matrix((data, (row, col)), shape=(n_vertices, n_vertices))

def diagnose_and_fix(rois, pts, polys, n_lh, min_size=50):
    n_vertices = pts.shape[0]
    n_rh = n_vertices - n_lh

    # split triangles LH/RH
    lh_tris = polys[polys.max(axis=1) < n_lh]
    rh_tris = polys[polys.min(axis=1) >= n_lh]

    # build adjacency
    adj_lh = mesh_adjacency(lh_tris, n_lh)
    adj_rh = mesh_adjacency(rh_tris - n_lh, n_rh)

    def process(mask, adj):
        idx = np.where(mask)[0]
        if len(idx) == 0:
            return {"n_comp": 0}, mask

        sub = adj[idx][:, idx]
        n_comp, labels = connected_components(sub, directed=False)

        sizes = np.bincount(labels)
        stats = {
            "n_comp": int(n_comp),
            "min": int(sizes.min()),
            "max": int(sizes.max()),
            "median": int(np.median(sizes)),
            "small_pct": float((sizes < min_size).sum() / n_comp * 100),
        }

        new_mask = mask.copy()
        for k in range(n_comp):
            if sizes[k] < min_size:
                new_mask[idx[labels == k]] = False

        return stats, new_mask

    fixed = {}
    diag = {}

    for roi_name, mask in rois.items():
        lh_mask = mask[:n_lh]
        rh_mask = mask[n_lh:]

        stats_lh, new_lh = process(lh_mask, adj_lh)
        stats_rh, new_rh = process(rh_mask, adj_rh)

        diag[roi_name] = {"lh": stats_lh, "rh": stats_rh}
        fixed[roi_name] = np.concatenate([new_lh, new_rh])

    return diag, fixed

In [4]:
subject = "sub-01"
rois_npz_fn = f"/Users/uriel/Downloads/{subject}_mmp_atlas_rois.npz"

rois_data = np.load(rois_npz_fn)
rois = {k: rois_data[k] for k in rois_data.keys()}

pts, polys = flatten_surf(cortex.db.get_surf(subject, "flat"))
n_lh = infer_nlh(polys)

diag, fixed_rois = diagnose_and_fix(rois, pts, polys, n_lh, min_size=50)

np.savez(f"/Users/uriel/Downloads/{subject}_mmp_atlas_rois_fixed.npz", **fixed_rois)
print("Done")

ValueError: negative axis 0 index: -191

In [14]:
cortex.db.get_surfinfo('sub-01')

<Vertex data for sub-01>